### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [29]:
from modelModule.model3 import VAE3
import torch

In [32]:
model = VAE3()
a = torch.randn((10,57))
b = torch.randn((10,57))
x, y, z = model(a,b)

### dataloader验证

In [1]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [2]:
dataset = FlatDataset('Letter_train.csv')
print(dataset.Max_Val)
print(dataset.Min_Val)

[15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
dataset = FlatDataset('Letter_train.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [10]:
test_data = imputed_data * global_max + global_min # 广播效应

In [27]:
a = torch.randn(2,3,4)
b = torch.nn.AdaptiveMaxPool1d(output_size=1)(a).squeeze(-1)

In [28]:
b.shape

torch.Size([2, 3])

### 模型精度测定

In [5]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
from modelModule.model2 import VAE2
from torch.utils.data import DataLoader
from utils import restore_data, result_show, get_missing
import torch
import numpy as np

In [6]:
## 模型参数载入
model = VAE2(dim=16, nhead=4)
checkpoint_path = r'模型参数保存\Letter_lr_0_0001_lr_scheduler_CosineAnnealingLR\version_3\checkpoints\epoch=49-step=3150.ckpt'    
data_norm = 'mean_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
train_csv = r'Letter_train.csv'
test_csv = r'Letter_test.csv'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(train_csv, data_norm=data_norm)
model.get_global_min_max(global_max=train_dataset.Max_Val, global_min=train_dataset.Min_Val)
print('载入成功')

载入成功


In [ ]:
test_dataset = FlatDataset(test_csv, data_norm=data_norm)
src_data = np.stack([test_dataset[index]['src_data'] for index in range(10)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(10)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = src_data.copy()
input[miss_matrix == 0] = np.nan
imputed_data, output = model.inference(input, miss_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
res_imputed_data = np.around(restore_data(imputed_data, train_dataset.Max_Val, train_dataset.Min_Val))

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))